### Portail meteo.data.gouv.fr - Données climatiques quotidienne (SIM2 = SAFRAN-ISBA) - Extraction de série chronologique pour une maille (1x1 km)

Auparavant, télécharger les données depuis le portail ci-dessous (chaque décennie repésente 1.1 Go en archive et 5 Go décompressé)<br>
1) Lecture du fichier de la décennie voulue
2) Extraction du point voulu et tracé de la carte de situation
3) Tracé du graphique chronologique de la décennie pour les paramètres et le point de maille choisis (par exemple 10 paramètres pour limiter l'occupation en mémoire vive)
4) Sauvegarde des séries chronologiques avec le graphique dans un fichier Excel<br>
    Le graphique dynamique est également sauvegardé en Html

data: https://meteo.data.gouv.fr/<br>

Auteur: https://github.com/loicduffar<br>

#### 1) Lecture du fichier CSV
Chaque fichier contient une décennie de données journalières depuis 1958 (à part le premier et le dernier dont les décennies sont incomplètes)<br>
La taille d'un fichier est donc très importante (5 Go environ), et le temps de lecture est environ de 13 minutes pour 10 paramètres

 - Renseignez le fichier de la décennie à lire (et son chemin d'accès)
 - Définir les paramètres désirés (par exemple 10 sous peine de satuer une mémoire vive de 8 Go)

ATENTION: La personalisation du point à lire est à faire ultérieurement dans la 2ème cellule (pour permettre d'interroger plusieurs points sans relire le fichier)

In [1]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12

# Lecture du fichier au format suivant dans un dataframe pandas

# LAMBX;LAMBY;DATE;PRENEI_Q;PRELIQ_Q;T_Q;FF_Q;Q_Q;DLI_Q;SSI_Q;HU_Q;EVAP_Q;ETP_Q;PE_Q;SWI_Q;DRAINC_Q;RUNC_Q;RESR_NEIGE_Q;RESR_NEIGE6_Q;HTEURNEIGE_Q;HTEURNEIGE6_Q;HTEURNEIGEX_Q;SNOW_FRAC_Q;ECOULEMENT_Q;WG_RACINE_Q;WGI_RACINE_Q;TINF_H_Q;TSUP_H_Q
# 600;24010;20200101;0.0;0.4;9.9;2.1;7.414;3161.6;111.1;98.4;0.1;0.3;0.3;0.940;2.2;0.1;0.0;0.0;0.000;0.000;0.000;0.0;0.0;0.315;0.000;9.5;10.9
# 600;24010;20200102;0.0;0.6;11.3;5.5;7.396;2955.5;55.2;89.8;0.7;0.4;-0.1;0.933;2.0;0.0;0.0;0.0;0.000;0.000;0.000;0.0;0.0;0.314;0.000;9.5;11.9
# 600;24010;20200103;0.0;0.8;9.6;6.1;6.458;3066.2;56.5;87.2;1.0;0.7;-0.2;0.926;1.9;0.1;0.0;0.0;0.000;0.000;0.000;0.0;0.0;0.313;0.000;8.8;10.6
# Etc…

import pandas as pd
import os
import datetime

# ================ Personalisation ====================
# Chemin d'accès au fichier d'entrée des données quotidiennes
folder_in= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\reference\SIM2"
# Chemin d'accès au fichier d'entrée des métadonnées de coordonnées des mailles SIM2
fld_meta= r'X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\reference\SIM2\metada'
# Chemin d'accès aux fichiers de sortie
folder_out= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\reference\SIM2"

# Nom du fichier CSV de métadonnées de coordonnées des mailles SIM2
file_meta= 'coordonnees_grille_safran_lambert-2-etendu.csv'
# Nom du fichier CSV de données à lire
file_name= 'QUOT_SIM2_latest-20231101-20231212.csv' # novembre 2023 jusqu'au jour précédent le teléchargement
file_name= 'QUOT_SIM2_2010_2019.csv'
file_name= 'QUOT_SIM2_latest-2020-202311.csv' # janvier 2020 jusqu'au mois précédent le teléchargement

# Colonnes à lire (pour économiser la mémoire vive) - Les 3 premières sont obligatoires (LAMBX, LAMBY, DATE)
usecols= ['LAMBX', 'LAMBY', 'DATE', 'PRENEI_Q', 'PRELIQ_Q', 'T_Q', 'FF_Q', 'Q_Q', 'DLI_Q', 'SSI_Q', 'HU_Q', 'EVAP_Q', 'ETP_Q', 'PE_Q', 'SWI_Q', 'DRAINC_Q', 'RUNC_Q', 'RESR_NEIGE_Q', 'RESR_NEIGE6_Q', 'HTEURNEIGE_Q', 'HTEURNEIGE6_Q', 'HTEURNEIGEX_Q', 'SNOW_FRAC_Q', 'ECOULEMENT_Q', 'WG_RACINE_Q', 'WGI_RACINE_Q', 'TINF_H_Q', 'TSUP_H_Q']

usecols= ['LAMBX', 'LAMBY', 'DATE', 'PRENEI_Q', 'PRELIQ_Q', 'T_Q',                'DLI_Q',          'HU_Q',           'ETP_Q',          'SWI_Q',                       'RESR_NEIGE_Q',                                                                                                                                 'TINF_H_Q', 'TSUP_H_Q']

# ================ Initialisation ====================
# Associe aux paramètres une unité et un nom long grâce à un dictionnaire (tiré de liste_parametres.odt https://www.data.gouv.fr/fr/datasets/r/d1ffaf5e-7d15-4fb5-a34c-f76aaf417b46)
dict_units= {'Precip': ['mm', 'Précipitations totales (06-06 UTC)'], 'PRENEI_Q': ['mm', 'Précipitations solides (06-06 UTC)'], 'PRELIQ_Q': ['mm', 'Précipitations liquides (06-06 UTC))'], 
             'T_Q': ['°C','Température moyenne'], 'FF_Q': ['m/s', 'Vit. vent'], 'Q_Q': ['g/kg','Humidité spécifique '], 'DLI_Q': ['J/cm2', 'Rayonnement atmosphérique '],
             'SSI_Q': ['J/cm2', 'Rayonnement visible '], 'HU_Q': ['%', 'Humidité relative '], 'EVAP_Q': ['mm', 'ETR (cumul quotidien 06-06 UTC)'], 
             'ETP_Q': ['mm', 'ETP (Penman-Monteith)'], 'PE_Q': ['mm', 'Pluies efficaces'], 'SWI_Q': ['%', 'Indice humidité des sols (06-06 UTC)'],
             'DRAINC_Q': ['mm', 'Drainage (06-06 UTC)'], 'RUNC_Q': ['mm', 'Ruissellement (06-06 UTC)'], 'RESR_NEIGE_Q': ['mm', 'Equivalent eau manteau neigeux (06-06 UTC)'], 
             'RESR_NEIGE6_Q': ['mm', 'Equivalent eau manteau neigeux à 06 UTC'], 'HTEURNEIGE_Q': ['m', 'Epaisseur manteau neigeux (moyenne 06-06 UTC)'], 
             'HTEURNEIGE6_Q': ['m', 'Epaisseur du manteau neigeux à 06 UTC)'], 'HTEURNEIGEX_Q': ['m', 'Epaisseur manteau neigeux maximum dans journée'], 
             'SNOW_FRAC_Q': ['%', 'Fraction maille recouverte par neige (moyenne 06-06 UTC)'], 'ECOULEMENT_Q': ['mm', 'Ecoulement en base manteau neigeux'], 
             'WG_RACINE_Q': ['mm','Contenu en eau liquide dans couche racinaire à 06 UTC'], 'WGI_RACINE_Q': ['mm', 'Contenu en eau gelée dans la couche de racinaire à 06 UTC'], 
             'TINF_H_Q': ['°C', 'Température minimale des 24 valeurs horaires'], 'TSUP_H_Q': ['°C', 'Température maximale des 24 valeurs horaires']}

# ================ Lecture des données quotidiennes ====================
# afficher l'heure 
start = datetime.datetime.now()
print("Heure de démarrage : ", start.strftime("%Y-%m-%d %H:%M"))
print("Attendez l'affichage des données et soyez patient ! La lecture prend environ 13 minutes pour une décennie entière et 10 paramètres...")

file_path = os.path.join(folder_in, file_name)
# Lit un fichier csv comportant des données météo dans un dataframe pandas, en précisant les champs à lire
df = pd.read_csv(file_path, sep=';', header=0, parse_dates=True, decimal='.', usecols= usecols)

# Ajoute une colonne 'ID' avec la concaténation des 2 colonnes  LAMBX et LAMBY
df['ID'] = df['LAMBX'].astype(str) + '_' + df['LAMBY'].astype(str)
df.drop(['LAMBX', 'LAMBY'], axis=1, inplace=True)
# index sur les 2 colonnes 'DATE' et 'ID'
df['DATE']= pd.to_datetime(df["DATE"].values, format='%Y%m%d').values

# df = df.set_index(['ID', 'DATE'])
df = df.set_index(['ID', 'DATE'])
print(df.columns)
# afficher la durée d'éxécution
now = datetime.datetime.now()
print("Heure de fin : ", now.strftime("%Y-%m-%d %H:%M"))
# affiche la différence entre les instants de début et de fin
print("Durée d'éxécution : ", now - start)

# ================ lit le fichier csv de métadonnées des coordonnées lat long des mailles
file_path = os.path.join(fld_meta, file_meta)

# Lit un fichier csv comportant des données météo dans un dataframe pandas
df_meta = pd.read_csv(file_path, sep=';', header=0, decimal=',')
# Ajoute une colonne 'ID' avec la concaténation des 2 colonnes  LAMBX et LAMBY
df_meta['ID'] = df_meta['LAMBX (hm)'].astype(str) + '_' + df_meta['LAMBY (hm)'].astype(str)
# supprime les colonnes 'LAMBX (hm)' et 'LAMBY (hm)'
df_meta.drop(['LAMBX (hm)', 'LAMBY (hm)'], axis=1, inplace=True)
# index sur la colonne 'ID'
df_meta = df_meta.set_index(['ID'], drop=True)
# display(df_meta)

# jointure des 2 dataframes en plaçant les nouvelles colonnes au début
df = df_meta.join(df, how='inner')

df

Heure de démarrage :  2023-12-23 11:32
Attendez l'affichage des données et soyez patient ! La lecture prend environ 13 minutes pour une décennie entière et 10 paramètres...
Index(['PRENEI_Q', 'PRELIQ_Q', 'T_Q', 'DLI_Q', 'HU_Q', 'ETP_Q', 'SWI_Q',
       'RESR_NEIGE_Q', 'TINF_H_Q', 'TSUP_H_Q'],
      dtype='object')
Heure de fin :  2023-12-23 11:36
Durée d'éxécution :  0:03:17.161348


LAT_DG   LON_DG  PRENEI_Q  PRELIQ_Q   T_Q   DLI_Q  \
ID          DATE                                                             
600_24010   2020-01-01  48.3822 -4.96118       0.0       0.4   9.9  3161.6   
            2020-01-02  48.3822 -4.96118       0.0       0.6  11.3  2955.5   
            2020-01-03  48.3822 -4.96118       0.0       0.8   9.6  3066.2   
            2020-01-04  48.3822 -4.96118       0.0       0.0   8.1  2564.4   
            2020-01-05  48.3822 -4.96118       0.0       0.0   7.2  2774.9   
...                         ...      ...       ...       ...   ...     ...   
11960_17450 2023-11-26  42.4619  9.57413       0.0       0.0   7.9  2403.7   
            2023-11-27  42.4619  9.57413       0.0       0.1  10.4  2558.1   
            2023-11-28  42.4619  9.57413       0.0       0.5  13.9  2535.8   
            2023-11-29  42.4619  9.57413       0.0       0.0  10.5  2527.6   
            2023-11-30  42.4619  9.57413       0.0       0.0  13.4  2920.4   

                        HU_Q  ETP_Q  SWI_Q  RESR_NEIGE_Q  TINF_H_Q  TSUP_H_Q  
ID          DATE                                                              
600_24010   2020-01-01  98.4    0.3  0.940           0.0       9.5      10.9  
            2020-01-02  89.8    0.4  0.933           0.0       9.5      11.9  
            2020-01-03  87.2    0.7  0.926           0.0       8.8      10.6  
            2020-01-04  78.8    0.8  0.919           0.0       5.4      10.2  
            2020-01-05  81.9    1.1  0.912           0.0       7.6       8.8  
...                      ...    ...    ...           ...       ...       ...  
11960_17450 2023-11-26  41.6    3.0  0.099           0.0       4.3      14.7  
            2023-11-27  63.4    1.7  0.097           0.0       6.1      16.0  
            2023-11-28  45.6    4.1  0.089           0.0      10.2      17.0  
            2023-11-29  57.8    2.1  0.085           0.0       6.7      16.2  
            2023-11-30  74.4    1.4  0.083           0.0       9.2      19.2  

[14145560 rows x 12 columns]

#### 2) Extrait les lignes de la maille voulue (maille la plus proches des coordonnées lat lon)
- Définir le point à lire

In [2]:
# Extrait les lignes de la maille voulue (maille la plus proches des coordonnées lat lon)
import plotly.express as px
import plotly.graph_objects as go

# ================ Personalisation ====================
# coordonnées du point pour lequel extraire les données
lat, lon= [43.529910019461035, 5.424477733642604]# Aix Galice
lat, lon= [44.70307665682509, 6.600304257413277] # St Crépin

# ================ Traitement ====================
# ----------- Initialisation
# duplique le dataframe
df_temp = df.copy(deep=True)

# calcule la distance entre chaque ligne et les coordonnées lat lon
df_temp['distance'] = ((df['LAT_DG'] - lat)**2 + (df['LON_DG'] - lon)**2)**0.5
# sélectionne la ligne avec la distance minimale
minimum = df_temp['distance'].min()

df_temp = df_temp.loc[df_temp['distance'] == minimum]

# supprime la colonne 'distance'
df_temp.drop(['distance'], axis=1, inplace=True)

# -----------trace une carte plotly avec le point de maille en spécifiant le nom de la trace pour la légende   

fig = px.scatter_mapbox(lat= [df_temp['LAT_DG'][0]], lon= [df_temp['LON_DG'][0]], mapbox_style= "open-street-map", 
                        title= 'Maille SAFRAN la plus proche des coordonnées fournies', height= 500, width= 700,
                        color_discrete_sequence= [ 'red'], size= [1 for i in [1]], size_max= 10,
                        # labels= 'Maille la plus proche'
                       hover_name= ['Maille la plus proche'], 
                      )

# Ajoute les coordonnées cibles avec le nom de la trace en légente
fig.add_trace(go.Scattermapbox(lat= [lat], lon= [lon], mode= 'markers', marker= {'size': 10, 'color': 'blue'}, name= 'Coordonnées cibles', ))

# sauvegarde la carte au format png et html
map_file= "SIM2_map"
fig.write_image(os.path.join(folder_out, map_file + '.png'))
fig.write_html(os.path.join(folder_out, map_file + '.html'))

# ----------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
display(df_temp)
fig.show()

2023-12-23 11:38


LAT_DG   LON_DG  PRENEI_Q  PRELIQ_Q  T_Q   DLI_Q  \
ID         DATE                                                            
9400_19770 2020-01-01  44.7103  6.62745       0.0       0.0  0.9  2095.4   
           2020-01-02  44.7103  6.62745       0.0       0.0  0.1  2044.7   
           2020-01-03  44.7103  6.62745       0.0       0.0 -0.1  2079.5   
           2020-01-04  44.7103  6.62745       0.0       0.0  0.5  1996.4   
           2020-01-05  44.7103  6.62745       0.0       0.0  0.4  2066.9   
...                        ...      ...       ...       ...  ...     ...   
           2023-11-26  44.7103  6.62745       0.0       0.0 -3.9  1947.1   
           2023-11-27  44.7103  6.62745       0.9       0.6 -0.9  2401.1   
           2023-11-28  44.7103  6.62745       0.0       0.0 -1.8  2362.9   
           2023-11-29  44.7103  6.62745      18.1       0.0 -3.8  2082.1   
           2023-11-30  44.7103  6.62745       2.1      60.1  0.5  2967.5   

                       HU_Q  ETP_Q  SWI_Q  RESR_NEIGE_Q  TINF_H_Q  TSUP_H_Q  
ID         DATE                                                              
9400_19770 2020-01-01  54.9    1.1  0.936          69.0      -1.8       7.5  
           2020-01-02  55.7    1.0  0.929          68.5      -3.0       5.8  
           2020-01-03  58.1    1.0  0.923          68.2      -2.7       5.3  
           2020-01-04  52.3    1.2  0.917          67.9      -3.3       6.5  
           2020-01-05  57.6    1.1  0.912          67.5      -2.9       6.8  
...                     ...    ...    ...           ...       ...       ...  
           2023-11-26  49.4    1.1  0.861           0.0     -10.4       3.8  
           2023-11-27  61.5    0.9  0.855           0.2      -5.3       3.7  
           2023-11-28  68.9    0.8  0.852           0.6      -2.5       1.9  
           2023-11-29  59.8    0.9  0.847           3.3     -10.4       2.4  
           2023-11-30  94.5    0.4  0.947          16.6      -2.5       1.9  

[1430 rows x 12 columns]

#### 3) Trace le graphique chronologique des paramètres

In [3]:
# Affiches des subplots multiples plotly superposé de tous des paramètres
# (température par exemple car pour les précipitations il faut choisir liquide ou solide ou en faire la somme)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ================ Traitement ====================
param_list= df_temp.columns.values[2:]
# Création d'un graphique plotly
fig = make_subplots(rows= len(param_list), cols= 1, shared_xaxes= True, vertical_spacing= 0.02, subplot_titles= param_list)

for i, param in enumerate(param_list):
    fig.add_trace(go.Scatter(x= df_temp.index.get_level_values(1).values, y= df_temp[param], name= param), row= i+1, col= 1)

    fig.update_yaxes(title_text= dict_units[param][0], row= i+1, col=1)
    fig.update_xaxes(title_text='Date', row= i+1, col= 1)
    fig.update_traces(line= dict(width=1), row= i+1, col= 1)
    # if i == len(param_list)-1:
    #     fig.update_xaxes(showticklabels=True, title_text='Date', row= i+1, col=1)
        
fig.update_layout(height= 1500, width= 1000, title_text= 'Paramètres météorologiques quotidiens SIM2 (SAFRAN)', title_x= 0.5,
                  hovermode='x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'))
# Trace une ligne verticale au travers tous les subplots matérialisant l'abscisse survolée par la souris 
xlast= 'x' + str(len(param_list))
fig.update_traces(xaxis= xlast)
fig.update_xaxes(showticklabels= True, title_text= 'Date', row= len(param_list), col= 1)

# définit le titre des différents subplots
for i, param in enumerate(param_list):
    fig.layout.annotations[i].update(text= dict_units[param][1])

# supprime la légende
fig.update_layout(showlegend=False)
# sauvegarde le graphique dans des fichiers image et html
graph_file= "SIM2_graph"
fig.write_image(os.path.join(folder_out, graph_file + ".png"))
fig.write_html(os.path.join(folder_out, graph_file + ".html"))

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
fig.show()

2023-12-23 11:38


#### 5) Sauvegarde les données et le graphique dans un fichier Excel

In [4]:
# Enregistre le dataframe dans un fichier excel

# ================ Personalisation ====================
# Chemin d'accès aux fichiers de sortie

# Nom du fichier CSV à lire

name_end= os.path.splitext(file_name)[0][-10:]

file_excel= 'QUOT_SIM2_' + name_end + '.xlsx'

# ================ Traitement ====================
file_path = os.path.join(folder_out, file_excel)
# Enregistre le dataframe dans un fichier excel
writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
workbook  = writer.book
df_temp.to_excel(writer, sheet_name='data', startrow=3)
worksheet = writer.sheets['data']
worksheet.write('A1', 'Lambert2 =')
worksheet.write('A2', 'Lat. Lon. ')

worksheet.write('B1', df_temp.index.get_level_values(0)[0])
worksheet.write('B2', str(lat) + '_' + str(lon))

# créé un autre sheet avec les graphiques
worksheet = workbook.add_worksheet('graphiques')
if os.path.exists(os.path.join(folder_out, graph_file + '.png')):
    worksheet.insert_image('A1', os.path.join(folder_out, graph_file + '.png'))
if os.path.exists(os.path.join(folder_out, map_file + '.png')):
    worksheet.insert_image('Q1', os.path.join(folder_out, map_file + '.png'))
workbook.close()

if os.path.exists(os.path.join(folder_out, graph_file + '.png')):
    os.remove(os.path.join(folder_out, graph_file + '.png'))
if os.path.exists(os.path.join(folder_out, map_file + '.png')):
    os.remove(os.path.join(folder_out, map_file + '.png'))

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
print('Enregistrement terminé du fichier excel : ', file_path)

2023-12-23 11:39
Enregistrement terminé du fichier excel :  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\reference\SIM2\QUOT_SIM2_020-202311.xlsx
